# PlaceKitten Demo Notebook 🐱

Interactive demonstration of PlaceKitten's Phase 2 features:
- Computer Vision Pipeline
- Smart Cropping Engine
- Step Visualization System
- Filter Pipeline

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display
import numpy as np

# Add src to path
project_root = Path(os.getcwd()).parent.parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Create output folder for notebook images
output_folder = Path("notebook_output")
output_folder.mkdir(exist_ok=True)

# Import PlaceKitten
from placekitten import PlaceKitten, list_available_filters

print("✅ PlaceKitten imported successfully!")
print(f"📁 Project root: {project_root}")
print(f"📁 Source path: {src_path}")
print(f"📁 Output folder: {output_folder.absolute()}")

# Helper function to get output path
def get_output_path(filename):
    return str(output_folder / filename)

## 1. Basic PlaceKitten Setup

In [ ]:
# Initialize PlaceKitten
kitten = PlaceKitten("demo")

# Show available images
images = kitten.list_available_images()
count = kitten.get_image_count()

print(f"🐱 Found {count} kitten images:")
for i, img in enumerate(images, 1):  # Show with 1-based indexing
    print(f"  {i}: {img}")

# Show available filters
filters = list_available_filters()
print(f"\n🎨 Available filters: {', '.join(filters)}")

## 2. New Flexible Features Demo

PlaceKitten now supports:
- **Optional dimensions**: Specify width only, height only, both, or neither
- **Aspect ratio preservation**: When one dimension is given, the other is calculated
- **Smart random selection**: Invalid or missing image_id automatically uses random
- **1-based indexing**: More intuitive numbering (1=first image, not 0)

In [ ]:
# Generate images using new flexible features

# 1. Full size image (no dimensions specified)
full_size = kitten.generate()
full_output = full_size.save(get_output_path("notebook_full_size.jpg"))
print(f"✅ Full size image: {full_size.get_size()}")

# 2. Width only - height calculated from aspect ratio
width_only = kitten.generate(width=600, image_id=1)  # 1-based indexing
width_output = width_only.save(get_output_path("notebook_width_only.jpg"))
print(f"✅ Width only (600px): {width_only.get_size()}")

# 3. Height only - width calculated from aspect ratio  
height_only = kitten.generate(height=400, image_id=1)  # Same image
height_output = height_only.save(get_output_path("notebook_height_only.jpg"))
print(f"✅ Height only (400px): {height_only.get_size()}")

# 4. Random image selection (no image_id)
random_img = kitten.generate(width=500, height=300)
random_output = random_img.save(get_output_path("notebook_random.jpg"))
print(f"✅ Random image: {random_img.get_size()}")

# Display examples
from IPython.display import HTML
display(HTML(f"""
<div style="display: flex; flex-wrap: wrap; gap: 20px;">
    <div>
        <h4>Full Size</h4>
        <img src="{full_output}" style="max-width: 200px; height: auto;">
        <p>Size: {full_size.get_size()}</p>
    </div>
    <div>
        <h4>Width Only (600px)</h4>
        <img src="{width_output}" style="max-width: 200px; height: auto;">
        <p>Size: {width_only.get_size()}</p>
    </div>
    <div>
        <h4>Height Only (400px)</h4>
        <img src="{height_output}" style="max-width: 200px; height: auto;">
        <p>Size: {height_only.get_size()}</p>
    </div>
    <div>
        <h4>Random Image</h4>
        <img src="{random_output}" style="max-width: 200px; height: auto;">
        <p>Size: {random_img.get_size()}</p>
    </div>
</div>
"""))

## 3. Aspect Ratio Preservation Demo

Demonstrate how PlaceKitten preserves aspect ratios when scaling:

In [ ]:
# Test aspect ratio preservation with same image
test_image_id = 1  # Use first image for consistency

# Get original dimensions
original = kitten.generate(image_id=test_image_id)
orig_w, orig_h = original.get_size()
orig_ratio = orig_w / orig_h

print(f"📐 Original image: {orig_w}x{orig_h}")
print(f"📐 Original aspect ratio: {orig_ratio:.3f}")

# Scale by width only
scaled_by_width = kitten.generate(width=800, image_id=test_image_id)
scale_w_size = scaled_by_width.get_size()
scale_w_ratio = scale_w_size[0] / scale_w_size[1]

print(f"📐 Scaled by width (800px): {scale_w_size[0]}x{scale_w_size[1]}")
print(f"📐 New aspect ratio: {scale_w_ratio:.3f}")

# Scale by height only
scaled_by_height = kitten.generate(height=600, image_id=test_image_id)
scale_h_size = scaled_by_height.get_size()
scale_h_ratio = scale_h_size[0] / scale_h_size[1]

print(f"📐 Scaled by height (600px): {scale_h_size[0]}x{scale_h_size[1]}")
print(f"📐 New aspect ratio: {scale_h_ratio:.3f}")

# Check preservation
ratio_diff_w = abs(orig_ratio - scale_w_ratio)
ratio_diff_h = abs(orig_ratio - scale_h_ratio)

if ratio_diff_w < 0.01 and ratio_diff_h < 0.01:
    print("✅ Aspect ratios perfectly preserved!")
else:
    print(f"❌ Aspect ratio drift: {ratio_diff_w:.4f}, {ratio_diff_h:.4f}")

# Save examples
orig_output = original.save(get_output_path("aspect_original.jpg"))
width_output = scaled_by_width.save(get_output_path("aspect_width_scaled.jpg"))
height_output = scaled_by_height.save(get_output_path("aspect_height_scaled.jpg"))

# Visual comparison
from IPython.display import HTML
display(HTML(f"""
<div style="display: flex; gap: 20px; align-items: flex-start;">
    <div style="text-align: center;">
        <h4>Original</h4>
        <img src="{orig_output}" style="max-width: 200px; border: 2px solid #ccc;">
        <p>{orig_w}x{orig_h}<br>Ratio: {orig_ratio:.3f}</p>
    </div>
    <div style="text-align: center;">
        <h4>Width Scaled (800px)</h4>
        <img src="{width_output}" style="max-width: 200px; border: 2px solid #00f;">
        <p>{scale_w_size[0]}x{scale_w_size[1]}<br>Ratio: {scale_w_ratio:.3f}</p>
    </div>
    <div style="text-align: center;">
        <h4>Height Scaled (600px)</h4>
        <img src="{height_output}" style="max-width: 200px; border: 2px solid #f00;">
        <p>{scale_h_size[0]}x{scale_h_size[1]}<br>Ratio: {scale_h_ratio:.3f}</p>
    </div>
</div>
"""))

## 4. Smart Random Selection Demo

Test how PlaceKitten handles invalid/missing image IDs:

In [ ]:
# Test smart random selection
print("🎲 Testing smart random selection...")

# Valid image ID (should work normally)
valid_img = kitten.generate(width=300, height=200, image_id=1)
valid_output = valid_img.save(get_output_path("random_valid.jpg"))
print(f"✅ Valid ID (1): {valid_img.get_size()}")

# Invalid image ID (should fall back to random - no error!)
try:
    invalid_img = kitten.generate(width=300, height=200, image_id=999)
    invalid_output = invalid_img.save(get_output_path("random_invalid.jpg"))
    print(f"✅ Invalid ID (999) → random fallback: {invalid_img.get_size()}")
except Exception as e:
    print(f"❌ Should not error: {e}")

# No image ID (should use random)
no_id_img = kitten.generate(width=300, height=200)
no_id_output = no_id_img.save(get_output_path("random_none.jpg"))
print(f"✅ No ID → random: {no_id_img.get_size()}")

# Force random selection
force_random = kitten.generate(width=300, height=200, image_id=1, random_selection=True)
force_output = force_random.save(get_output_path("random_forced.jpg"))
print(f"✅ Forced random (overrides ID): {force_random.get_size()}")

print("\n🎉 No errors! Invalid IDs gracefully fall back to random selection.")

# Display random examples
from IPython.display import HTML
display(HTML(f"""
<div style="display: flex; gap: 15px; flex-wrap: wrap;">
    <div style="text-align: center;">
        <h4>Valid ID (1)</h4>
        <img src="{valid_output}" style="max-width: 150px; border: 2px solid #0f0;">
    </div>
    <div style="text-align: center;">
        <h4>Invalid ID (999) → Random</h4>
        <img src="{invalid_output}" style="max-width: 150px; border: 2px solid #f80;">
    </div>
    <div style="text-align: center;">
        <h4>No ID → Random</h4>
        <img src="{no_id_output}" style="max-width: 150px; border: 2px solid #80f;">
    </div>
    <div style="text-align: center;">
        <h4>Forced Random</h4>
        <img src="{force_output}" style="max-width: 150px; border: 2px solid #f08;">
    </div>
</div>
"""))

## 5. Smart Cropping with Computer Vision

This demonstrates the 9-step intelligent cropping process:
1. Original analysis
2. Grayscale conversion
3. Noise reduction (Gaussian blur)
4. Edge detection (Canny algorithm)
5. Contour analysis
6. Subject bounding box
7. Rule of thirds grid
8. Optimal crop area
9. Final result

In [ ]:
# Test all available filters
filters_to_test = ['grayscale', 'sepia', 'blur', 'invert', 'pixelate']
filter_results = {}

base_processor = kitten.generate(width=400, height=300, image_id=3)  # Now 1-based indexing

# Generate original
original_file = base_processor.save(get_output_path("notebook_original.jpg"))
filter_results['original'] = original_file

# Apply each filter
for filter_name in filters_to_test:
    try:
        filtered_processor = base_processor.apply_filter(filter_name)
        output_file = filtered_processor.save(get_output_path(f"notebook_{filter_name}.jpg"))
        filter_results[filter_name] = output_file
        print(f"✅ {filter_name} filter applied")
    except Exception as e:
        print(f"❌ {filter_name} filter failed: {e}")
        filter_results[filter_name] = None

print(f"\n📊 Generated {len([f for f in filter_results.values() if f])} filter variations")

In [ ]:
# Display filter results in a grid
available_filters = [(k, v) for k, v in filter_results.items() if v and os.path.exists(v)]
n_filters = len(available_filters)

if n_filters > 0:
    cols = 3
    rows = (n_filters + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
    fig.suptitle('PlaceKitten Filter Pipeline Results', fontsize=16)
    
    # Handle single row case
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    
    for i, (filter_name, filepath) in enumerate(available_filters):
        row = i // cols
        col = i % cols
        
        ax = axes[row][col] if rows > 1 else axes[col]
        
        img = mpimg.imread(filepath)
        ax.imshow(img)
        ax.set_title(filter_name.title(), fontsize=12)
        ax.axis('off')
    
    # Hide empty subplots
    for i in range(n_filters, rows * cols):
        row = i // cols
        col = i % cols
        ax = axes[row][col] if rows > 1 else axes[col]
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No filter results to display")

## 6. Method Chaining Demo

Demonstrate the fluent interface with method chaining:

In [ ]:
# Complex method chaining example
chained_result = (
    kitten.generate(width=800, height=600, image_id=1)  # Now 1-based indexing
    .smart_crop(width=600, height=400, save_steps=False)
    .apply_filter("sepia")
    .save(get_output_path("notebook_chained.jpg"))
)

print(f"✅ Method chaining completed: {chained_result}")
display(Image(chained_result))

# Show the power of chaining
print("\n🔗 This single chain performed:")
print("   1. Generated 800x600 image from kitten #1")
print("   2. Smart cropped to 600x400 with computer vision")
print("   3. Applied sepia filter")
print("   4. Saved the final result")

## 7. Batch Processing Demo

In [ ]:
# Batch processing configuration
batch_configs = [
    {"width": 400, "height": 300, "filter_type": "grayscale", "image_id": 1},  # Now 1-based
    {"width": 500, "height": 281, "filter_type": "sepia", "image_id": 2},
    {"width": 600, "height": 400, "filter_type": "blur", "image_id": 3},
]

# Process batch in output folder
batch_results = kitten.batch_process(batch_configs, str(output_folder / "batch_output"))

print(f"✅ Batch processing completed: {len(batch_results)} images")
for i, result in enumerate(batch_results):
    print(f"   {i+1}. {result}")

# Display batch results
if batch_results:
    fig, axes = plt.subplots(1, len(batch_results), figsize=(15, 5))
    fig.suptitle('Batch Processing Results', fontsize=16)
    
    if len(batch_results) == 1:
        axes = [axes]
    
    for i, result_path in enumerate(batch_results):
        if os.path.exists(result_path):
            img = mpimg.imread(result_path)
            axes[i].imshow(img)
            config = batch_configs[i]
            title = f"{config['width']}x{config['height']}\n{config['filter_type']}"
            axes[i].set_title(title, fontsize=10)
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

## 8. Performance and Crop Information

In [ ]:
import time

# Performance test
start_time = time.time()

# Generate and process an image
processor = kitten.generate(width=1200, height=800, image_id=1)  # Now 1-based indexing
smart_processor = processor.smart_crop(width=1920, height=1080, save_steps=False)
result = smart_processor.save(get_output_path("notebook_performance_test.jpg"))

end_time = time.time()
processing_time = end_time - start_time

print(f"⏱️  Processing time: {processing_time:.2f} seconds")
print(f"📊 Generated 1920x1080 image with smart cropping")

# Show crop information if available
if hasattr(smart_processor, 'crop_info'):
    crop_info = smart_processor.crop_info
    print(f"\n🔍 Crop Information:")
    print(f"   Original size: {crop_info.get('original_size', 'N/A')}")
    print(f"   Target size: {crop_info.get('target_size', 'N/A')}")
    print(f"   Crop box: {crop_info.get('crop_box', 'N/A')}")
    print(f"   Subject bbox: {crop_info.get('subject_bbox', 'N/A')}")
    print(f"   Contour area: {crop_info.get('contour_area', 'N/A')}")

# Display final performance test result
if os.path.exists(result):
    display(Image(result))

## 9. Summary

🎉 **PlaceKitten Enhanced Features Complete!**

### ✅ **New Flexible Features**:
- **Optional dimensions**: width only, height only, both, or neither
- **Aspect ratio preservation**: Automatic scaling calculations
- **Smart random selection**: Graceful fallback for invalid image IDs
- **1-based indexing**: Intuitive numbering (1=first image)

### ✅ **Existing Advanced Features**:
- **Computer Vision Pipeline**: Edge detection, contour analysis, subject identification  
- **Smart Cropping Engine**: Rule-of-thirds, optimal positioning, boundary safety  
- **Step Visualization**: 9-step debug output with processing stages  
- **Filter Pipeline**: Multiple filters with method chaining  
- **Batch Processing**: Multiple image processing workflows  
- **Performance**: Fast processing with detailed crop information  

### 🔄 **Full Backward Compatibility**:
All existing code continues to work unchanged. The library is now more flexible while maintaining all original functionality.

The PlaceKitten library is ready for Phase 3 (Advanced Features) and Phase 4 (MCP Integration)!

In [ ]:
# Cleanup generated files (optional)
import glob

# List all generated files in output folder
generated_files = list(output_folder.glob("notebook_*")) + list(output_folder.glob("*batch_output*"))
print(f"📁 Generated {len(generated_files)} files during this demo:")
for f in sorted(generated_files):
    print(f"   📄 {f.name}")

print(f"\n📁 All files saved in: {output_folder.absolute()}")

# Uncomment to clean up:
# for f in generated_files:
#     try:
#         f.unlink()
#         print(f"🗑️  Removed {f.name}")
#     except:
#         pass